In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import sys
RunPath = os.getcwd() + "/"
CrysDatPath = "../CrysDat_FCC/" #"/home/sohamc2/HEA_FCC/MDMC/CrysDat_FCC/"
DataPath = "../MD_KMC_single/" #"/home/sohamc2/HEA_FCC/MDMC/ML_runs/DataSets/"
ModulePath = "../Symm_Network/" #"/home/sohamc2/VKMC/SymNetworkRuns/CE_Symmetry/Symm_Network/"

In [3]:
import numpy as np
import h5py
import torch as pt
from tqdm import tqdm
import pickle
from GCNetRun import Load_Data, makeComputeData, makeDataTensors
from GCNetRun import Load_crysDats, SpecBatchOuts, vacBatchOuts, sort_jp

In [4]:
T = 1073
state1List, state2List, dispList, rateList, AllJumpRates_st1,\
AllJumpRates_st2, avgDisps_st1, avgDisps_st2 = Load_Data(DataPath + "singleStep_{}_AllRates.h5".format(T))

found permuation


In [5]:
GpermNNIdx, NNsiteList, JumpNewSites, dxJumps = Load_crysDats(1, CrysDatPath)

z = dxJumps.shape[0]

RtoSiteInd = np.load(CrysDatPath + "RtoSiteInd.npy")
SiteIndToR = np.load(CrysDatPath + "SiteIndtoR.npy")

with open(CrysDatPath + "supercellFCC.pkl", "rb") as fl:
    superFCC = pickle.load(fl)
N_units = superFCC.superlatt[0,0]
N_units

8

# First test occupancy tensors

In [6]:
specCheck = 5
specsToTrain = [specCheck]
VacSpec = 0
N_check = 200
AllJumps = False

State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2, sp_ch =\
makeComputeData(state1List, state2List, dispList, specsToTrain, VacSpec, rateList,
                AllJumpRates_st1, AllJumpRates_st2, avgDisps_st1, avgDisps_st2, JumpNewSites,
                dxJumps, NNsiteList, N_check, AllJumps=AllJumps, mode="train")

Building Occupancy Tensors for species : [5]
No. of jumps : 200


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1452.07it/s]


In [7]:
State1_occs.shape

(200, 5, 512)

In [8]:
for samp in tqdm(range(N_check), position=0, leave=True):
    for site in range(state1List.shape[1]):
        if site == 0:
            assert np.all(State1_occs[samp, :, site] == 0)
        else:
            spec1 = state1List[samp, site]
            assert State1_occs[samp, spec1 - 1, site] == 1
            assert np.sum(State1_occs[samp, :, site]) == 1
            
            spec2 = state2List[samp, site]
            assert State2_occs[samp, spec2 - 1, site] == 1
            assert np.sum(State2_occs[samp, :, site]) == 1
        
    # check the displacements
    jSelect = None
    count = 0
    for jInd in range(JumpNewSites.shape[0]):
        state2_try = state1List[samp][JumpNewSites[jInd]]
        if np.all(state2_try == state2List[samp]):
            jSelect = jInd
            count += 1
    assert count == 1
        
    NNR = np.dot(np.linalg.inv(superFCC.crys.lattice), dxJumps[jSelect]).astype(int)
    assert np.allclose(np.dot(superFCC.crys.lattice, NNR), dxJumps[jSelect])
    NNRSite = superFCC.index(NNR, (0, 0))[0]
    
    NNsiteVac = NNsiteList[jSelect + 1, 0]
    assert np.all(NNRSite == NNsiteVac)
    
    spec = state1List[samp, NNsiteVac]
    assert np.allclose(disps[samp, 0], 3.59*dxJumps[jSelect])
    if spec == specCheck:
        assert np.allclose(disps[samp, 1], -3.59*dxJumps[jSelect])
    else:
        assert np.allclose(disps[samp, 1], 0.0)
    
    # check the rate
    assert np.math.isclose(rates[samp], rateList[samp])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 45.68it/s]


## Check the batched calculation of y vectors

### Make Data tensors

In [9]:
state1Data, state2Data, dispData, rateData, On_st1, On_st2 =\
makeDataTensors(State1_occs, State2_occs, rates, disps, OnSites_state1, OnSites_state2,
                specsToTrain, VacSpec, sp_ch, Ndim=3)

Training Species : [5]


In [10]:
On_st1.shape, N_check

(torch.Size([200, 512]), 200)

### Now check output with jump channels

In [11]:
jProbs_st1_array = AllJumpRates_st1 / np.sum(AllJumpRates_st1, axis=1).reshape(-1, 1)
jProbs_st2_array = AllJumpRates_st2 / np.sum(AllJumpRates_st2, axis=1).reshape(-1, 1)

In [32]:
cases = ((True, True), (True, False), (False, True), (False, False))

c = cases[3]

jsort = c[0]
jSwitch = c[1]

In [33]:
NNsites = pt.tensor(NNsiteList)
NNsvac_st1 = NNsites[1:, 0].repeat(N_check, 1)
NNsvac_st2 = NNsites[1:, 0].repeat(N_check, 1)

jProbs_st1, jProbs_st2, NNsvac_st1, NNsvac_st2 =\
            sort_jp(NNsvac_st1, NNsvac_st2, jProbs_st1_array[:N_check],
                    jProbs_st2_array[:N_check], jumpSort=jsort)

batchStart = 0
batchEnd = batchStart + N_check
y = pt.rand(N_check, z, 3, state1List.shape[1]).double()

jp1_tensor = jProbs_st1[:N_check]
jp2_tensor = jProbs_st2[:N_check]

y1, y2 = SpecBatchOuts(y, y, On_st1, On_st2, jp1_tensor, jp2_tensor, NNsvac_st1, NNsvac_st2,
                       Boundary_train=True, jumpSwitch=jSwitch)

Jump Rates unsorted. Values will be non-symmetric if boundary-training is active.


In [34]:
for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(z, 3).double()
    y2_samp = pt.zeros(z, 3).double()
    for site in range(On_st1.shape[1]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck-1] == 1:
            assert On_st1[sampInd, site]
            y1_samp += y[sampInd - batchStart, :, :, site]
        
        else:
            assert not On_st1[sampInd, site]
        
        if occs2[specCheck-1] == 1:
            assert On_st2[sampInd, site]
            y2_samp += y[sampInd - batchStart, :, :, site]
        else:
            assert not On_st2[sampInd, site]
    
    y1_samp_sum = pt.zeros(3).double()
    y2_samp_sum = pt.zeros(3).double()
    
    # Sort the jumps by rates
    if jsort:
        jProbs_st1_sort = np.argsort(jProbs_st1_array[sampInd, :])
    else:
        jProbs_st1_sort = np.arange(jProbs_st1_array[sampInd].shape[0])
        
    for jRank, jInd in enumerate(jProbs_st1_sort):
        # Get the jump site and species in the jump site
        if not jsort:
            assert jRank == jInd
        if not jSwitch:
            prob_j_st1 = jp1_tensor[sampInd, jRank]
            y1_samp_sum += y1_samp[jRank] * prob_j_st1     
        else:
            jSite = NNsiteList[1 + jInd, 0]
            sp = state1List[sampInd, jSite]
            if sp in specsToTrain:
                assert On_st1[sampInd, jSite]
                prob_j_st1 = jp1_tensor[sampInd, jRank]
                y1_samp_sum += y1_samp[jRank] * prob_j_st1     
            else:
                assert not On_st1[sampInd, jSite]
    if jsort:
        jProbs_st2_sort = np.argsort(jProbs_st2_array[sampInd, :])
    else:
        jProbs_st2_sort = np.arange(jProbs_st2_array[sampInd].shape[0])
        
    for jRank, jInd in enumerate(jProbs_st2_sort):
        if not jsort:
            assert jRank == jInd
        
        if not jSwitch:
            prob_j_st2 = jp2_tensor[sampInd, jRank]
            y2_samp_sum += y2_samp[jRank] * prob_j_st2
        else:
            jSite = NNsiteList[1 + jInd, 0]
            sp = state2List[sampInd, jSite] 
            if sp in specsToTrain:
                assert On_st2[sampInd, jSite]
                prob_j_st2 = jp2_tensor[sampInd, jRank]
                y2_samp_sum += y2_samp[jRank] * prob_j_st2
            else:
                assert not On_st2[sampInd, jSite]
            
    assert pt.allclose(y1[sampInd - batchStart], y1_samp_sum)
    assert pt.allclose(y2[sampInd - batchStart], y2_samp_sum)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 56.44it/s]


In [35]:
jsort

False

In [36]:
jSwitch

False